## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 9
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_9


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 17.9 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Dallas Cowboys,8,2021,30.1,8.2,291.5,107.1,12.0,142.8,24.0,...,12.0,101.0,17.000,12.75,12.25,1.375,1.575269,0.124224,2.500,3.333333
1,Baltimore Ravens,8,2021,27.6,8.1,266.3,94.6,12.6,161.6,24.4,...,12.8,91.8,14.125,12.50,11.75,1.250,1.745562,0.081250,1.625,1.857143


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
X['games_played'].unique().tolist()

[8, 9]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
4,Arizona Cardinals,2021,week_9,0.273566,0.255897,0.445112,0.355328,0.332476
3,Los Angeles Rams,2021,week_9,0.319489,0.285388,0.350076,0.340476,0.323857
2,Tampa Bay Buccaneers,2021,week_9,0.259118,0.247275,0.332884,0.295580,0.283714


In [10]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
4,Arizona Cardinals,2021,week_9,0.273566,0.255897,0.445112,0.355328,0.332476
3,Los Angeles Rams,2021,week_9,0.319489,0.285388,0.350076,0.340476,0.323857
2,Tampa Bay Buccaneers,2021,week_9,0.259118,0.247275,0.332884,0.295580,0.283714
0,Dallas Cowboys,2021,week_9,0.271842,0.221985,0.295567,0.248926,0.259580
7,Buffalo Bills,2021,week_9,0.263799,0.219915,0.239033,0.299232,0.255495
5,Las Vegas Raiders,2021,week_9,0.209759,0.141149,0.279297,0.241210,0.217854
10,Cleveland Browns,2021,week_9,0.157065,0.068701,0.126414,0.228171,0.145088
14,Tennessee Titans,2021,week_9,0.135646,0.067184,0.094619,0.173299,0.117687
13,Cincinnati Bengals,2021,week_9,0.101636,0.057303,0.096016,0.197207,0.113041
1,Baltimore Ravens,2021,week_9,0.117525,0.056587,0.099643,0.174273,0.112007
